# Bouncing Droplet Simulation and Visualization

This notebook demonstrates the simulation of hydrophobic sphere impact and bouncing droplets.

Based on:
Galeano-Rios et al. (2017) "Non-wetting impact of a sphere onto a bath and its application to bouncing droplets"

In [2]:
using IJulia
IJulia.installkernel("Julia", "--project=@.")

┌ Info: Installing 'Julia 1.11.7' kernelspec in /Users/helloworld/Library/Jupyter/kernels/julia-1.11
└ @ IJulia /Users/helloworld/.julia/packages/IJulia/f10T5/deps/kspec.jl:125


"/Users/helloworld/Library/Jupyter/kernels/julia-1.11"

In [3]:
using BouncingDroplet
using Plots
using LinearAlgebra
using SparseArrays
using Printf

ERROR: LoadError: UndefVarError: `DirichletToNeumannOperator` not defined in `BouncingDroplet`
Stacktrace:
 [1] top-level scope
   @ ~/Home/Desktop/amugeona/2017_paper_replication/src/solver/matrix_assembly.jl:1
 [2] include(mod::Module, _path::String)
   @ Base ./Base.jl:562
 [3] include(x::String)
   @ BouncingDroplet ~/Home/Desktop/amugeona/2017_paper_replication/src/BouncingDroplet.jl:1
 [4] top-level scope
   @ ~/Home/Desktop/amugeona/2017_paper_replication/src/BouncingDroplet.jl:35
 [5] include
   @ ./Base.jl:562 [inlined]
 [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base ./loading.jl:2881
 [7] top-level scope
   @ stdin:6
in expression starting at /Users/helloworld/Home/Desktop/amugeona/2017_paper_replication/src/solver/matrix_assembly.jl:1
in expression starting at /Users/helloworld/Home/Desktop/amugeona/2

ErrorException: Failed to precompile BouncingDroplet [12345678-1234-1234-1234-123456789abc] to "/Users/helloworld/.julia/compiled/v1.11/BouncingDroplet/jl_MBYPk7".

## 1. Setup Physical Parameters

In [ ]:
# Water properties
fluid = water_properties()

println("Fluid properties:")
println("  ρ = $(fluid.ρ) kg/m³")
println("  σ = $(fluid.σ) N/m")
println("  ν = $(fluid.ν) m²/s")

In [ ]:
# Sphere properties
R₀ = 0.96e-3  # 0.96 mm (from Lee & Kim 2008)
ρ_s = 1320.0  # kg/m³
V₀ = 0.89     # m/s impact velocity

solid = SolidProperties(R₀=R₀, ρ_s=ρ_s)

println("\nSolid properties:")
println("  R₀ = $(R₀*1000) mm")
println("  ρ_s = $(ρ_s) kg/m³")
println("  m = $(solid.m*1e6) μg")
println("  V₀ = $(V₀) m/s")

In [ ]:
# Dimensionless numbers
params = dimensionless_sphere_impact(fluid, solid, V₀)

println("\nDimensionless parameters:")
println("  Re = $(round(params.Re, digits=1))")
println("  Fr = $(round(params.Fr, digits=1))")
println("  We = $(round(params.We, digits=2))")
println("  M = $(round(params.M, digits=1))")

## 2. Create Mesh and Operators

In [ ]:
# Create radial mesh
mesh = create_mesh_for_sphere(R₀, R₀*10, points_per_radius=40, domain_size_factor=10.0)

println("Mesh created:")
println("  Number of points: $(mesh.nr)")
println("  Radial spacing: $(mesh.δr*1e6) μm")
println("  Domain radius: $(mesh.R*1000) mm")

In [ ]:
# Build operators
println("\nBuilding operators...")
@time ΔH = build_laplacian_operator(mesh)
@time N_op = DirichletToNeumannOperator(mesh, polar_refinement=10, n_θ=128)

println("Operators built successfully")

## 3. Sphere Geometry

In [ ]:
# Create sphere
sphere = SphericalSolid(R₀)
z_s = evaluate_on_mesh(sphere, mesh)

# Plot sphere profile
valid_idx = findall(.!isnan.(z_s))
plot(mesh.r[valid_idx]*1000, z_s[valid_idx]*1000,
     xlabel="r [mm]", ylabel="z_s [mm]",
     title="Sphere Surface Profile",
     lw=2, legend=false)
hline!([0], ls=:dash, color=:black)

## 4. Initial Condition

In [ ]:
# Initial state (sphere above surface)
h₀ = 2*R₀  # Initial height of sphere bottom
v₀ = -V₀   # Downward velocity

state = initialize_impact_state(mesh, sphere, h₀, v₀)

println("Initial state:")
println("  h₀ = $(h₀*1000) mm")
println("  v₀ = $(v₀) m/s")
println("  Contact: $(state.k_contact > 0)")

## 5. Simulation (Placeholder)

**Note**: The full simulation requires a stable fluid solver. Current implementation is for demonstration of structure.

In [ ]:
# Simulation parameters
δt = 1e-5      # Time step
t_max = 0.01   # Maximum simulation time
n_steps = Int(t_max / δt)

println("Simulation setup:")
println("  δt = $(δt) s")
println("  t_max = $(t_max) s")
println("  Total steps = $(n_steps)")

## 6. Visualization Functions

In [ ]:
"""
Plot current state of simulation
"""
function plot_state(state, mesh, sphere, z_s)
    p1 = plot(mesh.r*1000, state.η*1000,
             xlabel="r [mm]", ylabel="η [mm]",
             title="Free Surface",
             lw=2, label="η(r)")
    
    # Add sphere position
    valid_idx = findall(.!isnan.(z_s))
    plot!(mesh.r[valid_idx]*1000, (state.h .+ z_s[valid_idx])*1000,
          lw=2, label="Sphere", color=:red)
    hline!([0], ls=:dash, color=:black, label="")
    
    p2 = plot(mesh.r*1000, state.φ,
             xlabel="r [mm]", ylabel="φ",
             title="Velocity Potential",
             lw=2, legend=false)
    
    p3 = plot(mesh.r*1000, state.ps,
             xlabel="r [mm]", ylabel="ps",
             title="Pressure",
             lw=2, legend=false)
    
    # Sphere trajectory info
    info_text = @sprintf("t = %.4f s\nh = %.2f mm\nv = %.2f m/s\nContact: %d points",
                        state.t, state.h*1000, state.ht, state.k_contact)
    
    p4 = plot(annotation=(0.5, 0.5, info_text), 
             framestyle=:none, title="Status")
    
    plot(p1, p2, p3, p4, layout=(2,2), size=(1000, 800))
end

In [ ]:
# Plot initial state
plot_state(state, mesh, sphere, z_s)

## 7. Analysis Functions

In [ ]:
"""
Compute forces on droplet
"""
function compute_forces(state, mesh, params)
    # Pressure force (placeholder)
    F_pressure = sum(state.ps) * mesh.δr * 2π
    
    # Gravity force
    F_gravity = -1/params.Fr
    
    # Total force
    F_total = F_pressure + F_gravity
    
    return (pressure=F_pressure, gravity=F_gravity, total=F_total)
end

## 8. Export Functions

In [ ]:
"""
Save simulation results
"""
function save_results(filename, history)
    # Placeholder for saving data
    println("Results would be saved to: $filename")
end

## Summary

This notebook provides the framework for:
- Setting up physical parameters
- Creating computational mesh
- Building differential operators
- Initializing simulation state
- Visualizing results

**Next Steps**:
1. Debug and stabilize fluid solver
2. Implement full contact solver
3. Run sphere impact simulations
4. Compare with experimental data (Lee & Kim 2008)
5. Implement bouncing droplet simulations